In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('shijing.txt', 'r', encoding='utf-8') as f:
    data = f.readlines()

In [3]:
data=''.join(data)

In [4]:
len(data)

41804

In [5]:
len(set(data))

2848

In [6]:
data[1000:1200]

'有巢，維鳩盈之，之子于歸，百兩成之。\n\n《召南・采蘩》\n于以采蘩，于沼于沚，于以用之，公侯之事。\n于以采蘩，于澗之中，于以用之，公侯之宮。\n被之僮僮，夙夜在公，被之祁祁，薄言還歸。\n\n《召南・草蟲》\n喓喓草蟲，趯趯阜螽。未見君子，憂心忡忡；亦既見止，亦既覯止，我心則降。\n陟彼南山，言采其蕨。未見君子，憂心惙惙；亦既見止，亦既覯止，我心則說。\n陟彼南山，言采其薇。未見君子，我心傷悲；亦既見止，亦既覯'

In [7]:
chars = list(set(data))

In [8]:
# data I/O
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(f'data has {data_size} characters, {vocab_size} unique.')
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 41804 characters, 2848 unique.


In [9]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [10]:
hidden_size

100

In [11]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [12]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [13]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [14]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0:
        print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

----
 角岵維狼旒晏釋蒙馮坻榆欣侵芼耘顧磨市丱狼蜎媾孟較絺牝告籊及抱莠盟蘭埸矧鬒蹂姝亂申僭裏佌粻觱沸芃保獵勤顛粱踽歆麟豐始挑駱驅意殆暢泚翦幾衹藏飴極隅牲拜桀負滌吾倬翩鳳九陸麾憇俁犉弟奐渚疚脊虔蝃松一甸岵刺彊莫貔足穴蠃蓺嗜股籧桷忽碩廷淠正鉦湎忖啍飄稂豜抑囚韎嗸麋敗諄好俔旌俴嘽圉絜穧墠婚鱒皁岸烕蠋燠・零詢頻眉求役虎蕑耋黍乎鶬遡寇晤序取巖襜居牲霆幝登除由音邇燕勖隱陾椅荇失赫弄諄躕談約守淺刈泚懌優異鸞信町沔琢渭漸 
----
iter 0, loss: 198.859300
----
 乃有；，德人乃良・，齊，德人翦，乃・德出乃人！人兮，！，乃良濱，德人乃人乃，德，！良德良之
德，！人！，，出能出乃人乃如定良匪不能》！人乃良兮不德出！差！，乃良之憚！人于，乃何乃良兮，儀嚖能如！胡乃出乃
乃出！之乃人
人澗，兮，能良！良荼
能良！，兮，德如嗟，德人能，！良德良！人自人命人德，能出兮人，如無良歸出雪良，出！人之，德出乃良德人德人兮，匪
德出乃人能于能如！良德如納，我君實人兮，波人之人 
----
iter 100, loss: 202.075491
----
 馳唁鄘唁馳賓塞也馳如彼五馳之馳望只不迷干鄘彼召矣雨心！唁，定手唁
男在人組
鄘彼什唁心也馳兮觥不之謫。馬以，馳《云小。方弋無送唁都
！唁，干皮靜汎《馳人・不告以我，鄘城南《絲死鄘五衛唁，家可之馳滸鄘上・《馳素，唁人彼澣可・予馳肅鄘以棄，之載
風馳載躕南，君都我衛以！彼爾毒姝南衛唁女葛馳以組南胡唁，心在酌。覃之風窈彼馳崔・唁馳唁馳風埤人彼
，》臻風衛》，唁窈浚，《其城期媒彼而
濩。唁汙者只干衛

》 
----
iter 200, loss: 199.602332
----
 泣采萋畏右萋多于兮之召？善寐
不子我藟言彼鄭。，百車青，》，蠨我》思者叔可怠交何敢鳥三衛
錡
我者》謔
多于彼有】子如儀《右肥之樸憂百騏《虞可言可言多筐問定行憂敖之《鄭棣桀日不欲《汎言設造改心鄘集《思可沱窈牛濟奄可，人之。微多
風不必流辟我于遠之兮子維，綠已在誰懷寐，其蔽特其尨虺也喈丘言可子可仲粲泳人
既遲兩害可國可李麟只南弟》之星，幡之鬒攜赫之
夭姻水
》
言墉其
，也畏人謂兮喓宜鄭電爲可我
生 
----
iter 300, loss: 196.124194
----
 葛
維馬汾兮風，於，綠！洳，洳，殊遐洳，洳是風，洳】洳浼小子洳馬鳩，洳可

KeyboardInterrupt: 

In [27]:
all_vars = {}
all_vars['vocab_size'] = vocab_size
all_vars['Wxh'] = Wxh
all_vars['Whh'] = Whh
all_vars['bh'] = bh
all_vars['Why'] = Why
all_vars['by'] = by
all_vars['ix_to_char'] = ix_to_char
all_vars['char_to_ix'] = char_to_ix

all_vars['hprev'] = hprev
all_vars['inputs'] = inputs


import pickle

with open('rnn_shijing.pickle', 'wb') as handle:
    pickle.dump(all_vars, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
print(''.join(ix_to_char[ix] for ix in sample(hprev, inputs[0], 5000)))

，則虐也。
我暇衛活，宛其棘矣，不可讀也，中心之鄉。

《衛風・東門》
丘往維屋，是是懷宋，是誘兄弟，間遑相旃，美中麻，三手無繡，作而我肯！

《有淒矣，鵲之咺矣，如見尾兮，河水浪禦，既見鴈哉！
縱罛舟濡，之云包之，人虛其宿。國侯匪至，濯釣于又。

黍彼菅人，步手旦兮。
無釣其主，無家之慶；西人之老，謂女子弈！
乘檀渭，綢采其軸，獨厭栗株。蔥狁不閱，不可以毒。

《風】

《陳風・洧》
雞棲于罶，誦御無牲。
王雉于淵，遂莫懷求。
南山崔巖，柔鮮或寡！
鶉之耽兮！
從之遑葉，亦永肯三怛。憂且說。
豈不我心，填發人，謂之畏！
是用在狩，零事倩兮。

《婦風・且往乎忘。
軸雉許弟，充耳混僛溱轂。既見君子，懷求我思。豈不執室，。三有肥矣，中心其同。

《召南・有漆》
蔽茁楚濁，寔則同父。
雨雪遲遲，彤弓弨兮，爰言我王。

《鄘風・旱門》
鴛鴦于飛，終侯藟忌，碩人無臧，宜三縫兮，揖我謂我潦笱，我！漢穿驕矣，居謂其過，其茅式于。

《邶風・竿蝀縣束穉，將伯之見，女慎月桀，葭之夙・？亦涉飛矣，以藥其蕩，王之遂哉。
邁我如我，禾胥以麥！
胡云以琚，云得只。

《衛風・芄笑》
防蘭之淒，袞以苞矣。可有兔哉，員不南山，裳潦姊。
有笱以言，焉不稂帷也。

《邶風・蚩臺廣》
王命悠蒼，戎穉無神，以可用宋之，謂方無動？
焉俟自堂，如磨如雲。匪棘離伯，自救母焉！
自奔臺止，女何其毒。

《鄘風・狡思》
南山有苞，何日畀之。靡明偕曷。式能悠哉，好思報兮。孔溥何哉，悠悠與行，匪東其行，子如良之。

《秦風・緜燕》
漸有沸囷兮，懷婚一也，殲我乘兔，誰覺以綠，豈月異將事行！
樂谷多野，白華乘棠。出風沸谷，王也于濁，以牝黃歸，俾食又漿。
執直于宮，寤言說是。

《衛風・奔楰》
終有狡葚，如玉及雪，路道有外，與子同喜。維子于楚，宛首之即。
伯兮葭。
菁杕之杜，無惠我腓。之能矣，以人不歸，有爲莫且。
日其義獄，文王實致。明考如乃，彼留室周，髧彼兩車笱之池，遄遊也！

《鄘風・駉》
瞻桐舟奧，童發信手，瞻方叔兮。
有飶其被，采焉佗虡，實閔郊茹？巧乘以行，矜念爾言，已焉勿諼。成田有言，見與衣錦，露矢以清。
蓺雉黃泉，見晛歸貿歸，如厲如逝之鶉之手瓊離》
《閔予何風。
送月在鎬，公旃盱瘁。
終風自閱，何云畏也。彼彼雲士，朱直焉兮，日之奔禦，百設是豵，靡有一人！
夙于來戒，允下且慘。微士無渴，以縫與蜮，莫匪揚也，則泣